<a href="https://colab.research.google.com/github/Codes-of-Hermit/Finance_codes/blob/main/Value_of_the_contract_Natural_Gas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from scipy.optimize import curve_fit

# --- 1. Price Model Dependency (From Previous Task) ---
def price_model(t, slope, intercept, amplitude, shift):
    """The seasonal price model function."""
    frequency = 2 * np.pi / 365.25
    return slope * t + intercept + amplitude * np.sin(frequency * t + shift)

def train_price_model(df):
    """Fits the model to historical data to get parameters."""
    df['Dates'] = pd.to_datetime(df['Dates'])
    start_date = df['Dates'].min()
    days_data = (df['Dates'] - start_date).dt.days
    price_data = df['Prices']

    # Initial guesses
    p0 = [0, np.mean(price_data), np.std(price_data), 0]
    params, _ = curve_fit(price_model, days_data, price_data, p0=p0)
    return params, start_date

def get_estimated_price(target_date, params, start_date_ref):
    """Predicts price for a specific date using the model."""
    if isinstance(target_date, str):
        target_date = pd.to_datetime(target_date)
    days_delta = (target_date - start_date_ref).days
    return price_model(days_delta, *params)


# --- 2. Contract Pricing Function ---

def calculate_contract_value(injection_dates, withdrawal_dates,
                             injection_rate, withdrawal_rate,
                             max_volume, storage_cost_per_month,
                             injection_withdrawal_cost_per_unit,
                             transport_cost_per_trip,
                             price_params, start_ref_date):
    """
    Calculates the net value of a natural gas storage contract.

    Args:
        injection_dates (list): List of tuples [(date_str, volume), ...]
        withdrawal_dates (list): List of tuples [(date_str, volume), ...]
        injection_rate (float): Max volume allowed per injection event.
        withdrawal_rate (float): Max volume allowed per withdrawal event.
        max_volume (float): Max total storage capacity.
        storage_cost_per_month (float): Fixed monthly rental fee.
        injection_withdrawal_cost_per_unit (float): Variable cost per MMBtu.
        transport_cost_per_trip (float): Fixed cost per transport event.
        price_params (list): Model parameters from train_price_model.
        start_ref_date (datetime): Reference start date from train_price_model.

    Returns:
        float: The estimated net value of the contract.
    """

    volume_in_storage = 0
    cash_flows = 0
    total_transport_costs = 0
    total_variable_costs = 0

    # Combined list of all event dates to determine contract duration
    all_dates = [pd.to_datetime(d[0]) for d in injection_dates] + \
                [pd.to_datetime(d[0]) for d in withdrawal_dates]

    if not all_dates:
        return 0.0

    # Calculate Storage Duration (in months)
    start_contract = min(all_dates)
    end_contract = max(all_dates)
    # Approximate duration in months
    duration_months = (end_contract - start_contract).days / 30.44
    total_storage_costs = storage_cost_per_month * duration_months

    # --- Process Injections ---
    for date_str, vol in injection_dates:
        # 1. Check Constraints
        if vol > injection_rate:
            print(f"Warning: Injection {vol} exceeds rate {injection_rate} on {date_str}")
        if volume_in_storage + vol > max_volume:
            print(f"Warning: Storage exceeded! Capacity: {max_volume}, Current: {volume_in_storage + vol}")

        # 2. Calculate Costs
        est_price = get_estimated_price(date_str, price_params, start_ref_date)
        cost_to_buy = est_price * vol

        # 3. Update Cash Flow & Inventory
        cash_flows -= cost_to_buy
        volume_in_storage += vol
        total_transport_costs += transport_cost_per_trip
        total_variable_costs += (vol * injection_withdrawal_cost_per_unit)

    # --- Process Withdrawals ---
    for date_str, vol in withdrawal_dates:
        # 1. Check Constraints
        if vol > withdrawal_rate:
            print(f"Warning: Withdrawal {vol} exceeds rate {withdrawal_rate} on {date_str}")
        if volume_in_storage - vol < 0:
            print(f"Warning: Withdrawal {vol} exceeds available volume {volume_in_storage}")

        # 2. Calculate Revenue
        est_price = get_estimated_price(date_str, price_params, start_ref_date)
        revenue_from_sell = est_price * vol

        # 3. Update Cash Flow & Inventory
        cash_flows += revenue_from_sell
        volume_in_storage -= vol
        total_transport_costs += transport_cost_per_trip
        total_variable_costs += (vol * injection_withdrawal_cost_per_unit)

    # --- Final Value Calculation ---
    net_value = cash_flows - total_storage_costs - total_transport_costs - total_variable_costs
    return net_value

# --- 3. Example Usage ---

# Load data and train model (Required to get params)
df = pd.read_csv('Nat_Gas.csv')
params, start_ref = train_price_model(df)

# Define Contract Terms
inputs = {
    'injection_dates': [('2024-06-15', 1000000)],  # Inject 1M MMBtu in Summer
    'withdrawal_dates': [('2024-12-15', 1000000)], # Withdraw 1M MMBtu in Winter
    'injection_rate': 2000000,       # Max volume per action
    'withdrawal_rate': 2000000,      # Max volume per action
    'max_volume': 5000000,           # Max storage capacity
    'storage_cost_per_month': 100000,# $100k/month fixed fee
    'injection_withdrawal_cost_per_unit': 0.01, # $0.01 per MMBtu ($10k per 1M)
    'transport_cost_per_trip': 50000 # $50k per injection/withdrawal
}

# Run Pricing Model
value = calculate_contract_value(
    inputs['injection_dates'], inputs['withdrawal_dates'],
    inputs['injection_rate'], inputs['withdrawal_rate'],
    inputs['max_volume'], inputs['storage_cost_per_month'],
    inputs['injection_withdrawal_cost_per_unit'],
    inputs['transport_cost_per_trip'],
    params, start_ref
)

print(f"Estimated Contract Value: ${value:,.2f}")

Estimated Contract Value: $476,407.89


/tmp/ipython-input-3089403739.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dates'] = pd.to_datetime(df['Dates'])
